In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


## Chi Mean hr, and sdnn, and rsmdd, in one code chunck

In [27]:
# Data for Chi meditation
df_chi_hrv = pd.read_csv("HRV_Chi_Meditation_Combined.csv")
df_chi_rr = pd.read_csv("RR_Chi_meditation_Combined.csv")
#print("The following are columns for chi hrv", df_chi_hrv.columns)


df_chi_pre_hrv = df_chi_hrv[df_chi_hrv['State'] == 'Pre_Meditation'] # specific for before meditation
df_chi_med_hrv = df_chi_hrv[df_chi_hrv['State'] == 'Meditation'] # specific for during meditation

df_chi_pre_rr = df_chi_rr[df_chi_rr['State'] == 'Pre_Meditation'] # specific for before meditation
df_chi_med_rr = df_chi_rr[df_chi_rr['State'] == 'Meditation']  # specific for during meditation

##df_chi_pre_hrv.head()
#print("Unique values for subject in chi_hrv\n:", df_chi_hrv["Subject"].unique())
#print("Unique values for subject in chi_rr\n:", df_chi_rr["Subject"].unique())

In [ ]:
# Mean hr, and sdnn, and rsmdd, in one code chunck
# reference for calculating sdnn and rmssd[11]. 
# Get a list of all unique subjects in chi_hrv
unique_subjects = df_chi_hrv["Subject"].unique()

# Store mean HR results
mean_hrv_chi_results = []

# Loop through each subject: for hrv
for subject in unique_subjects:
    # Filter data for that subject
    subject_data = df_chi_hrv[df_chi_hrv["Subject"] == subject]
    
    # Calculate mean heart rate
    mean_hr = subject_data["HeartRate"].mean()

    # Append to list
    mean_hrv_chi_results.append({"Subject": subject, 
                                "Mean_HR_Chi": mean_hr})
# Convert results to DataFrame
mean_hrv_df = pd.DataFrame(mean_hrv_chi_results)


results_chi_rr = []

# Loop over each unique subject: for rr
for subject, group in df_chi_rr.groupby("Subject"):
    rr_intervals = group["RR_Interval"].dropna().values

    # due to how rr data are supposed to be, any with less than
    # 2 values is likely to be a mistake
    # it is unlikely to be this way. But just for caution
    if len(rr_intervals) < 2:
        print(f"Skip {subject} due to insufficient RR data.")
        continue

    # SDNN: Standard deviation of NN (RR) intervals
    sdnn = np.std(rr_intervals, ddof=1)

    # RMSSD: Root mean square of successive RR differences
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    # Grab condition from the 'State' column (they should be all the same per group)
    condition = group["State"].iloc[0]  # 'Pre-Meditation' or 'Meditation'
    # this allow to not manually code the condition

    results_chi_rr.append({
        "Subject": subject,
        "State": condition,
        "SDNN": sdnn,
        "RMSSD": rmssd
    })

# Final DataFrame
chi_rr_metrics = pd.DataFrame(results_chi_rr)

#chi_rr_metrics["Subject_clean"] = chi_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

chi_rr_metrics.head()


Subject           State      SDNN     RMSSD
0  C1_med_rr      Meditation  0.048511  0.018704
1  C1_pre_rr  Pre-Meditation  0.119323  0.155018
2  C2_med_rr      Meditation  0.047154  0.055643
3  C2_pre_rr  Pre-Meditation  0.076216  0.067354
4  C3_med_rr      Meditation  0.040667  0.022403

In [29]:
#mean_hrv_df.head()

In [ ]:
# Combine mean_hr, rmssd and sdnn together.
# Step 1: Clean 'Subject' in chi_rr_metrics using .replace[12].

chi_rr_metrics["Subject_clean"] = chi_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

# Step 2: Merge on cleaned subject name
chi_combined = pd.merge(
    mean_hrv_df,
    chi_rr_metrics,
    left_on="Subject",
    right_on="Subject_clean",
    how="inner"
)

# Step 3: Drop unnecessary columns and rename
chi_combined = chi_combined.drop(columns=["Subject_clean", "Subject_y"])
chi_combined = chi_combined.rename(columns={
    "Subject_x": "Subject",
    "Mean_HR_Chi": "Mean_HR"
})

# Optional: Add group + label for ML
chi_combined["Group"] = "Chi"
chi_combined["Label"] = chi_combined["State"]

chi_combined.head()

Subject    Mean_HR           State      SDNN     RMSSD Group           Label
0  C1_pre  71.027691  Pre-Meditation  0.119323  0.155018   Chi  Pre-Meditation
1  C1_med  73.749095      Meditation  0.048511  0.018704   Chi      Meditation
2  C2_pre  83.985172  Pre-Meditation  0.076216  0.067354   Chi  Pre-Meditation
3  C2_med  82.396551      Meditation  0.047154  0.055643   Chi      Meditation
4  C3_pre  84.252398  Pre-Meditation  0.065371  0.026263   Chi  Pre-Meditation

### Data for ironman

In [31]:
# Data for ironman 
df_irm_hrv = pd.read_csv("HRV_Ironman_Meditation_Combined.csv")
df_irm_rr = pd.read_csv("RR_Ironman_Combined.csv")

print("Unique values for subject in df_irm_hrv\n:", df_irm_hrv["Subject"].unique())
print("Unique values for subject in df_irm_rr\n:", df_irm_rr["Subject"].unique())

Unique values for subject in df_irm_hrv
: ['I1_hrv' 'I2_hrv' 'I3_hrv' 'I4_hrv' 'I5_hrv' 'I6_hrv' 'I7_hrv' 'I8_hrv'
 'I9_hrv']
Unique values for subject in df_irm_rr
: ['I1_rr' 'I2_rr' 'I3_rr' 'I4_rr' 'I5_rr' 'I6_rr' 'I7_rr' 'I8_rr' 'I9_rr']


In [ ]:
# ---------- MEAN HR ----------
unique_subjects_irm = df_irm_hrv["Subject"].unique()
mean_hrv_irm_results = []

for subject in unique_subjects_irm:
    subject_data = df_irm_hrv[df_irm_hrv["Subject"] == subject]
    mean_hr = subject_data["HeartRate"].mean()
    
    # Hardcoding 'Sleeping' since there's no Pre/Post in Ironman
    condition = "Sleeping"

    mean_hrv_irm_results.append({
        "Subject": subject,
        "State": condition,
        "Mean_HR": mean_hr
    })

mean_hr_df_irm = pd.DataFrame(mean_hrv_irm_results)


# ---------- SDNN + RMSSD ----------
results_rr_irm = []

for subject_rr, group in df_irm_rr.groupby("Subject"):
    rr_intervals = group["RR_Interval"].dropna().values

    
    sdnn = np.std(rr_intervals, ddof=1)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    
    # Also hardcoding 'Meditation' here
    condition = "Sleeping"

    results_rr_irm.append({
        "Subject": subject_rr,
        "State": condition,
        "SDNN": sdnn,
        "RMSSD": rmssd
    })

irm_rr_metrics = pd.DataFrame(results_rr_irm)
#irm_rr_metrics.head()


In [33]:
#mean_hr_df_irm.head()

In [ ]:
## Combined Mean_HR and SDNN, RMSSD
# Step 1: Clean Subject names to match
mean_hr_df_irm["Subject_clean"] = mean_hr_df_irm["Subject"].str.replace("_hrv", "", regex=False)
irm_rr_metrics["Subject_clean"] = irm_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

# Step 2: Merge on Subject and State
ironman_combined = pd.merge(
    mean_hr_df_irm,
    irm_rr_metrics,
    on=["Subject_clean", "State"],
    suffixes=("_HR", "_RR")
)

# Step 3: Clean column names and reorder
ironman_combined = ironman_combined.drop(columns=["Subject_HR", "Subject_RR"])
ironman_combined = ironman_combined.rename(columns={"Subject_clean": "Subject"})

# Step 4: Add group and label (keep label = state)
ironman_combined["Group"] = "Ironman"
ironman_combined["Label"] = ironman_combined["State"]  # Keep as 'Sleeping'

# Step 5: Reorder columns
desired_order = ["Subject", "Group", "State", "Mean_HR", "SDNN", "RMSSD",  "Label"]
ironman_combined = ironman_combined[desired_order]

# Preview
ironman_combined.head()


Subject    Group     State    Mean_HR      SDNN     RMSSD     Label
0      I1  Ironman  Sleeping  42.976741  0.085527  0.082208  Sleeping
1      I2  Ironman  Sleeping  61.364071  0.038160  0.019082  Sleeping
2      I3  Ironman  Sleeping  54.193073  0.062127  0.065087  Sleeping
3      I4  Ironman  Sleeping  55.268795  0.046562  0.023236  Sleeping
4      I5  Ironman  Sleeping  45.612603  0.064177  0.046415  Sleeping

## Data for metron

In [35]:
# Data for metron
df_met_hrv = pd.read_csv("HRV_Metron_Combined.csv")
df_met_rr = pd.read_csv("RR_Metron_Combined.csv")

print("Unique values for subject in df_met_hrv\n:", df_met_hrv["Subject"].unique())
print("Unique values for subject in df_met_rr\n:", df_met_rr["Subject"].unique())

Unique values for subject in df_met_hrv
: ['M1_hrv' 'M2_hrv' 'M3_hrv' 'M4_hrv' 'M5_hrv' 'M6_hrv' 'M7_hrv' 'M8_hrv'
 'M9_hrv' 'M10_hrv' 'M11_hrv' 'M12_hrv' 'M13_hrv' 'M14_hrv']
Unique values for subject in df_met_rr
: ['M1_rr' 'M2_rr' 'M3_rr' 'M4_rr' 'M5_rr' 'M6_rr' 'M7_rr' 'M8_rr' 'M9_rr'
 'M10_rr' 'M11_rr' 'M12_rr' 'M13_rr' 'M14_rr']


In [ ]:
# ---------- MEAN HR ----------
unique_subjects_met = df_met_hrv["Subject"].unique()
mean_hrv_met_results = []

for subject in unique_subjects_met:
    subject_data = df_met_hrv[df_met_hrv["Subject"] == subject]
    mean_hr = subject_data["HeartRate"].mean()
    condition = "Fixed-Rate Breathing"  # Assumption

    mean_hrv_met_results.append({
        "Subject": subject,
        "State": condition,
        "Mean_HR": mean_hr
    })

mean_hr_df_met = pd.DataFrame(mean_hrv_met_results)

# ---------- SDNN + RMSSD ----------
results_rr_met = []

for subject_rr, group in df_met_rr.groupby("Subject"):
    rr_intervals = group["RR_Interval"].dropna().values

    
    sdnn = np.std(rr_intervals, ddof=1)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    condition = "Fixed-Rate Breathing"

    results_rr_met.append({
        "Subject": subject_rr,
        "State": condition,
        "SDNN": sdnn,
        "RMSSD": rmssd
    })

met_rr_metrics = pd.DataFrame(results_rr_met)

#met_rr_metrics.head(10)


In [37]:
#mean_hr_df_met.head()

In [ ]:
## Combine Mean_HR, SDNN, RMSDD
# Step 1: Clean Subject names
mean_hr_df_met["Subject_clean"] = mean_hr_df_met["Subject"].str.replace("_hrv", "", regex=False)
met_rr_metrics["Subject_clean"] = met_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

# Step 2: Merge on Subject and State
metron_combined = pd.merge(
    mean_hr_df_met,
    met_rr_metrics,
    on=["Subject_clean", "State"],
    suffixes=("_HR", "_RR")
)

# Step 3: Clean columns
metron_combined = metron_combined.drop(columns=["Subject_HR", "Subject_RR"])
metron_combined = metron_combined.rename(columns={"Subject_clean": "Subject"})

# Step 4: Add group and label (Label = State)
metron_combined["Group"] = "Metronomic"
metron_combined["Label"] = metron_combined["State"]  # e.g., Fixed-Rate Breathing

# Step 5: Reorder columns
desired_order = ["Subject", "State", "Mean_HR", "SDNN", "RMSSD", "Group", "Label"]
metron_combined = metron_combined[desired_order]

# Preview
#metron_combined.head()


## Data for normal

In [39]:
# Data for normal

df_nor_hrv = pd.read_csv("HRV_Normal_Combined.csv")
df_nor_rr = pd.read_csv("RR_Normal_Combined.csv")


print("Unique values for subject in df_nor_hrv\n:", df_nor_hrv["Subject"].unique())
print("Unique values for subject in df_nor_rr\n:", df_nor_rr["Subject"].unique())

Unique values for subject in df_nor_hrv
: ['N1_hrv' 'N2_hrv' 'N3_hrv' 'N4_hrv' 'N5_hrv' 'N6_hrv' 'N7_hrv' 'N8_hrv'
 'N9_hrv' 'N10_hrv' 'N11_hrv']
Unique values for subject in df_nor_rr
: ['N1_rr' 'N2_rr' 'N3_rr' 'N4_rr' 'N5_rr' 'N6_rr' 'N7_rr' 'N8_rr' 'N9_rr'
 'N10_rr' 'N11_rr']


In [ ]:
# ---------- MEAN HR ----------
unique_subjects_nor = df_nor_hrv["Subject"].unique()
mean_hrv_nor_results = []

for subject in unique_subjects_nor:
    subject_data = df_nor_hrv[df_nor_hrv["Subject"] == subject]
    mean_hr = subject_data["HeartRate"].mean()
    condition = "Sleeping"  # Assumption

    mean_hrv_nor_results.append({
        "Subject": subject,
        "State": condition,
        "Mean_HR": mean_hr
    })

mean_hr_df_nor = pd.DataFrame(mean_hrv_nor_results)

# ---------- SDNN + RMSSD ----------
results_rr_nor = []

for subject_rr, group in df_nor_rr.groupby("Subject"):
    rr_intervals = group["RR_Interval"].dropna().values

    
    sdnn = np.std(rr_intervals, ddof=1)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    condition = "Sleeping"

    results_rr_nor.append({
        "Subject": subject_rr,
        "State": condition,
        "SDNN": sdnn,
        "RMSSD": rmssd
    })

nor_rr_metrics = pd.DataFrame(results_rr_nor)

#nor_rr_metrics.head()


In [41]:
#mean_hr_df_nor.head()

In [ ]:
## Combine Mean_HR, SDNN, RMSDD
# Step 1: Clean Subject names
mean_hr_df_nor["Subject_clean"] = mean_hr_df_nor["Subject"].str.replace("_hrv", "", regex=False)
nor_rr_metrics["Subject_clean"] = nor_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

# Step 2: Merge on Subject and State
normal_combined = pd.merge(
    mean_hr_df_nor,
    nor_rr_metrics,
    on=["Subject_clean", "State"],
    suffixes=("_HR", "_RR")
)

# Step 3: Clean column names
normal_combined = normal_combined.drop(columns=["Subject_HR", "Subject_RR"])
normal_combined = normal_combined.rename(columns={"Subject_clean": "Subject"})

# Step 4: Add group and label (Label = State)
normal_combined["Group"] = "Normal"
normal_combined["Label"] = normal_combined["State"]  # Keep label as "Sleep"

# Step 5: Reorder columns
desired_order = ["Subject", "State", "Mean_HR", "SDNN", "RMSSD", "Group", "Label"]
normal_combined = normal_combined[desired_order]

# Preview
#normal_combined.head()


Subject     State    Mean_HR      SDNN     RMSSD   Group     Label
0      N1  Sleeping  60.050299  0.062894  0.056111  Normal  Sleeping
1      N2  Sleeping  60.907089  0.083574  0.100456  Normal  Sleeping
2      N3  Sleeping  62.384693  0.074912  0.058169  Normal  Sleeping
3      N4  Sleeping  70.577594  0.061155  0.051756  Normal  Sleeping
4      N5  Sleeping  47.907891  0.097930  0.093663  Normal  Sleeping

## Yoga Mean_HR, SDNN, RMSDD

In [43]:
# Data for yoga meditation
df_yog_hrv = pd.read_csv("HRV_Yoga_combined.csv")
df_yog_rr = pd.read_csv("RR_Yoga_Combined.csv")


# Split Yoga HRV data
df_yog_pre_hrv = df_yog_hrv[df_yog_hrv['State'] == 'Pre_Meditation']
df_yog_med_hrv = df_yog_hrv[df_yog_hrv['State'] == 'Meditation']

# Split Yoga RR data
df_yog_pre_rr = df_yog_rr[df_yog_rr['State'] == 'Pre_Meditation']
df_yog_med_rr = df_yog_rr[df_yog_rr['State'] == 'Meditation']

print("Unique values for subject in df_yog_hrv\n:", df_yog_hrv["Subject"].unique())
print("Unique values for subject in df_yog_rr\n:", df_yog_rr["Subject"].unique())



Unique values for subject in df_yog_hrv
: ['Y1_pre_hrv' 'Y1_med_hrv' 'Y2_pre_hrv' 'Y2_med_hrv' 'Y3_pre_hrv'
 'Y3_med_hrv' 'Y4_pre_hrv' 'Y4_med_hrv']
Unique values for subject in df_yog_rr
: ['Y1_med_rr' 'Y1_pre_rr' 'Y3_med_rr' 'Y3_pre_rr' 'Y4_med_rr' 'Y4_pre_rr']


In [ ]:
# Extract unique subjects from HRV data
unique_subjects = df_yog_hrv["Subject"].unique()
mean_hrv_yog_results = []

for subject in unique_subjects:
    subject_data = df_yog_hrv[df_yog_hrv["Subject"] == subject]
    mean_hr = subject_data["HeartRate"].mean()
    condition = subject_data["State"].iloc[0] # because there is more than one

    mean_hrv_yog_results.append({
        "Subject": subject,
        "State": condition,  
        "Mean_HR": mean_hr
    })

# Convert to DataFrame
mean_hr_df_yog = pd.DataFrame(mean_hrv_yog_results)



results_rr_yog = []

for subject_rr, group in df_yog_rr.groupby("Subject"):
    rr_intervals = group["RR_Interval"].dropna().values
    
    sdnn = np.std(rr_intervals, ddof=1)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    condition = group["State"].iloc[0]
    


    

    results_rr_yog.append({
        "Subject": subject_rr,
        "State": condition,
        "SDNN": sdnn,
        "RMSSD": rmssd
    })

# Convert to DataFrame
yog_rr_metrics = pd.DataFrame(results_rr_yog)


In [45]:
#yog_rr_metrics.head()

In [46]:
#mean_hr_df_yog.head()

In [ ]:
# Combine together
# Step 1: Clean Subject columns in both DataFrames
mean_hr_df_yog["Subject_clean"] = mean_hr_df_yog["Subject"].str.replace("_hrv", "", regex=False)
yog_rr_metrics["Subject_clean"] = yog_rr_metrics["Subject"].str.replace("_rr", "", regex=False)

# Step 2: Merge on Subject + State
yoga_combined = pd.merge(
    mean_hr_df_yog,
    yog_rr_metrics,
    on=["Subject_clean", "State"],
    suffixes=("_HR", "_RR")
)

# Step 3: Clean column names
yoga_combined = yoga_combined.drop(columns=["Subject_HR", "Subject_RR"])
yoga_combined = yoga_combined.rename(columns={"Subject_clean": "Subject"})

# Step 4: Add group + label (KEEP LABEL AS-IS)
yoga_combined["Group"] = "Yoga"
yoga_combined["Label"] = yoga_combined["State"]  # keep consistent labels

# Step 5: Reorder columns
desired_order = ["Subject","Group", "State", "Mean_HR", "SDNN", "RMSSD", "Label"]
yoga_combined = yoga_combined[desired_order]

# Preview
#yoga_combined.head()



Subject Group           State    Mean_HR      SDNN     RMSSD           Label
0  Y1_pre  Yoga  Pre-Meditation  64.567786  0.022046  0.019831  Pre-Meditation
1  Y1_med  Yoga      Meditation  98.752263  0.032485  0.010809      Meditation
2  Y3_pre  Yoga  Pre-Meditation  66.512289  0.036194  0.025202  Pre-Meditation
3  Y3_med  Yoga      Meditation  81.333017  0.046082  0.026529      Meditation
4  Y4_pre  Yoga  Pre-Meditation  45.394962  0.103781  0.112275  Pre-Meditation
5  Y4_med  Yoga      Meditation  88.410841  0.081636  0.078944      Meditation

### Combine everything together

In [ ]:
#Combine all final DataFrames
all_hrv_rr_metric_combined = pd.concat([
    chi_combined,
    yoga_combined,
    ironman_combined,
    metron_combined,
    normal_combined
], ignore_index=True)

# Preview
all_hrv_rr_metric_combined.head()


Subject    Mean_HR           State      SDNN     RMSSD Group           Label
0  C1_pre  71.027691  Pre-Meditation  0.119323  0.155018   Chi  Pre-Meditation
1  C1_med  73.749095      Meditation  0.048511  0.018704   Chi      Meditation
2  C2_pre  83.985172  Pre-Meditation  0.076216  0.067354   Chi  Pre-Meditation
3  C2_med  82.396551      Meditation  0.047154  0.055643   Chi      Meditation
4  C3_pre  84.252398  Pre-Meditation  0.065371  0.026263   Chi  Pre-Meditation

#### Save as a csv file

In [49]:
# remove the comment to run it. But otherwise run it only once.
#all_hrv_rr_metric_combined.to_csv("all_hrv_rr_metric_combined.csv", index=False)


In [50]:
all_hrv_rr_metric_combined['State'].unique()

array(['Pre-Meditation', 'Meditation', 'Sleeping', 'Fixed-Rate Breathing'],
      dtype=object)